In [41]:
import pandas as pd
import numpy as np
from dateutil.parser import parse
from datetime import datetime, timedelta
import math

samplesize = 250

In [42]:
from textblob import TextBlob #sentiment analysis
import nltk
nltk.download('punkt')



def _gender(data):
    try:
        if data and data.lower()=='male': return 1
        else: return 0
    except: return -1
    
def _age(data):
    try:
        if data and str(data) !='nan':
            bd = parse(data)
            diff = datetime.now() - bd
            return diff.days 
        else: return 0
    except Exception as e: 
        print (e)
        print (data)
        return -1
    
def _nationality(data):
    if data and 'singapore' in data.lower(): return 1   
    else: return 0
    
def _resi_status(data):
    if data and  data.lower() == 'citizen' or  data.lower() == 'pr': return 1   
    else: return 0
    
def _yesno(data):
    if data:
        if str(data).lower() == 'yes' : return 1 
        else: return 0
    else: return -1
    
def _occupation(data):
    if data and 'tour' in str(data).lower(): return 1   
    else: return 0
    
def _self_employ_time(data):
    import calendar
    try:
        if data and str(data) != 'nan':
            
            
            _m = list(calendar.month_abbr).index(data.split(";")[0]) 

            

            __y = data.split(";")[1].replace('Before','').replace(' ','') # get rid of silly "Before"
            _y = int(__y)

            #print ( (datetime.now() -   datetime(_y,_m,1)).days)
            return (datetime.now() -   datetime(_y,_m,1)).days
            #else: 
            #    ym = data.split(';')[2]
            #    _y = int(ym.split("-")[0])
            #    _m = int(ym.split("-")[1])
            #    return (datetime.now() -   datetime(_y,_m,1)).days
                
        else: return 0
    except Exception as e: 
        print ('mthyear exception')
        print (data)
        print (e)
        return -10
    

    
def _log(data):
    try:
        #print (para)
        if data and float(data)>0:return math.log10(float(data))
        else: return 0
    except Exception as e: 
        #print (e)
        return -1
def _int(data):
    if data and str(data) != 'nan': return int(data)
    else: return -1
    
    


import statistics
def _sentiment_of(text):
    if text:
        blob = TextBlob(str(text))
        _sentiments = []
        for sentence in blob.sentences:
            _sentiments.append(sentence.sentiment.polarity * sentence.sentiment.subjectivity)
        return  statistics.mean(_sentiments) 
    else: return -10
    
def _interested(data):
    if data and 'contact' in str(data): return 1
    else: return 0
    
def _evaluate_checker(data):
    if data and str(data) != 'nan':
        try:
            _checker = data.split(";")[0] 
            if _checker != '' and str(_checker) != 'nan' and str(_checker) == 'ESCALATE' : return -1
            elif _checker == '' or str(_checker) == 'nan':
                _checker = data.split(";")[1]
                
                
                if _checker != '' and str(_checker) != 'nan' and str(_checker) == 'ESCALATE': return -1
                elif _checker == '' or str(_checker) == 'nan': return -10
                else: return float(_checker)
            
            
            else:  return float(_checker)
                
            
        except Exception as e: 
            print (e)
            return -10
        
    else: return -10
    
def _f(n):
    return str(n)

def _label_outcome(data):
    if data:
        if str(data) == 'Approve - 1000': return 1000
        elif str(data) == 'Approve - 800': return 800
        elif str(data) == 'Approve - 400': return 400
        else: return 0
    else: return -1

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mengweetan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [43]:


df = pd.read_csv('output2.csv', dtype='unicode') # this is the final file!


#_result = df[0:samplesize]
_result = df
print ('start')
old_columns = [i for i in _result.columns]
#print (old_columns)

_result['_gender']=_result['Gender'].apply(_gender)
_result['_age']=_result['Date of Birth'].apply(_age)
_result['_nationality']=_result['Nationality'].apply(_nationality)
_result['_resi_status']=_result['Residential Status'].apply(_resi_status)

_result = pd.concat([_result,pd.get_dummies(_result['Housing Type'], prefix='onehot')],axis=1)
_result['_hdb_registered_address']=_result['Registered Address'].apply(_yesno)
_result = pd.concat([_result,pd.get_dummies(_result['Highest Education'], prefix='onehot')],axis=1)
_result['_occupation']=_result['Occupation'].apply(_occupation)
#_result['_av']=_result['Annual Value (AV) of  registered address'].apply(_av,args=(-1,))
_result['_av_value']=_result['Annual Value (AV) of  registered address'].apply(_log)

_result['__temp']=_result['Self Employed Since (Month)']+';'+_result['Self Employed Since (Year)']
_result['_self_employ_time']=_result['__temp'].apply(_self_employ_time)


        
print ('50%') 
_result['_2018_t']=_result['Income in 2018 (NOA YA2019) - Trade income'].apply(_log)
_result['_2018_e']=_result['\xa0Income in 2018 (NOA YA2019) - Employment income'].apply(_log)
_result['_2019_e']=_result['Income in 2019 (NOA YA2020) - Trade income'].apply(_log)
_result['_2019_t']=_result['Income in 2019 (NOA YA2020) - Employment income'].apply(_log)
_result['_2020_e']=_result['Income in 2020 (Jan-Mar) - Trade income'].apply(_log)
_result['_2020_t']=_result['Income in 2020 (Jan-Mar) - Employment income'].apply(_log)
_result['s_2019_i']=_result['Spouse\'s Income in 2019'].apply(_log)
_result['s_2020_i']=_result['Spouse\'s Income in 2020 (Jan-Mar)'].apply(_log)

_result['_child']=_result['No. of Singaporean Children < 21'].apply(_int)
_result['_parent']=_result['No. of Singaporean Parents > 64'].apply(_int)
print ('60%') 
_result = pd.concat([_result,pd.get_dummies(_result['Marital Status'], prefix='onehot')],axis=1)
_result['_married']=_result['Marriage Date'].apply(_age)
_result['_divorced']=_result['Divorce Date'].apply(_age)

_result['_hdb']=_result['Number of HDB properties own'].apply(_int)
_result['_pte']=_result['Number of private properties own'].apply(_int)
_result['_comz']=_result['Number of commercial properties own'].apply(_int)
_result['_comz_av']=_result['Annual Value (AV) of commercial properties own'].apply(_log)
_result['_comz_operate']=_result['Biz Operations from  commercial properties'].apply(_yesno)
print ('80%') 

_result['_hardship_sentiment']=_result['Please describe hardship (if any)'].apply(_sentiment_of)
print ('90%') 
_result['_ntuc_covid']=_result['Interested in NTUC Covid19 Programme'].apply(_interested)

#_result['__temp2']=_result['Checker Outcome\nValue'].str +';'+_result['Checker Outcome\nNew'].str  # checker!
_result['Checker Outcome\nValue'] = _result['Checker Outcome\nValue'].apply(_f) 
_result['Checker Outcome\nNew'] = _result['Checker Outcome\nNew'].apply(_f) 
_result['__temp2']= _result[['Checker Outcome\nValue', 'Checker Outcome\nNew']].agg(';'.join, axis=1)
_result['_checker']=_result['__temp2'].apply(_evaluate_checker)

_result['LABEL']=_result['Final Outcome Combined'].apply(_label_outcome)


_result = _result.drop(old_columns,axis=1)
print ('end')

_result.to_csv ('_final.csv', index = False, header=True)

start
50%
60%
80%
90%
end


In [46]:
from machine import Machine

In [49]:
m = Machine()


(72669, 49)
(72669, 4)
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 15)                750       
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 64        
Total params: 814
Trainable params: 814
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
m.train()

Train on 58135 samples, validate on 14534 samples
Epoch 1/150
58135/58135 [==============================] - 4s 73us/sample - loss: 104.6483 - accuracy: 0.6070 - val_loss: 2.3729 - val_accuracy: 0.6912
Epoch 2/150
58135/58135 [==============================] - 4s 64us/sample - loss: 4.9305 - accuracy: 0.6416 - val_loss: 1.8985 - val_accuracy: 0.6973
Epoch 3/150
58135/58135 [==============================] - 4s 66us/sample - loss: 4.0866 - accuracy: 0.6391 - val_loss: 2.9888 - val_accuracy: 0.6775
Epoch 4/150
58135/58135 [==============================] - 4s 65us/sample - loss: 3.0514 - accuracy: 0.6432 - val_loss: 1.1793 - val_accuracy: 0.5740
Epoch 5/150
58135/58135 [==============================] - 4s 72us/sample - loss: 2.3669 - accuracy: 0.6443 - val_loss: 1.6400 - val_accuracy: 0.5641
Epoch 6/150
58135/58135 [==============================] - 5s 84us/sample - loss: 2.2997 - accuracy: 0.6427 - val_loss: 1.2103 - val_accuracy: 0.6883
Epoch 7/150
58135/58135 [=======================

In [7]:
import tensorflow as tf
load_model = tf.keras.models.load_model
model = load_model('_modelv2.h5',custom_objects={'tf': tf}, compile=False)


In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 15)                735       
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 64        
Total params: 799
Trainable params: 799
Non-trainable params: 0
_________________________________________________________________


In [12]:
import numpy as np
input = [1,16748,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0,507,0.0,0.0,2.24551266781415,4.3448440193371205,2.552668216112193,3.5531545481696254,4.47567118832443,3.845098040014257,1,1,0,1,0,0,6005,0,1,0,0,0.0,0,0.026813888888888894,1,1000.0] #1000
_i = np.reshape(input,(len(input),1))

_a = model.predict([_i.T])
print (_a)
np.argmax(_a)  #0 = 0,1=400, 2=800, 3 = 1k

[[7.07054138e-01 1.09936984e-04 3.91905494e-02 2.53645360e-01]]


0